# M5 Walmart Sales Forecasting with Prophet

This notebook performs EDA on the M5 dataset and trains Facebook Prophet models for demand forecasting.

Dataset Location: `ml_models/datasets/m5-forecasting-accuracy/`

Goals:
1. Exploratory Data Analysis (EDA)
2. Data preprocessing for single store
3. Prophet model training
4. Forecast evaluation (MAPE, accuracy)

## Part 1: Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Prophet for forecasting
from prophet import Prophet

# Metrics
from sklearn.metrics import mean_absolute_percentage_error

# Styling
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print('All libraries imported successfully')

In [ ]:
# Define dataset path
DATASET_PATH = Path('../../datasets/m5-forecasting-accuracy')

# Check if files exist
sales_file = DATASET_PATH / 'sales_train.csv'
calendar_file = DATASET_PATH / 'calendar.csv'
prices_file = DATASET_PATH / 'sell_prices.csv'

print(f'Sales data file exists: {sales_file.exists()}')
print(f'Calendar file exists: {calendar_file.exists()}')
print(f'Prices file exists: {prices_file.exists()}')

if not sales_file.exists():
    print('\nWARNING: Dataset files not found!')
    print(f'Expected location: {DATASET_PATH.absolute()}')

In [ ]:
# Load datasets
print('Loading M5 datasets...')

sales_df = pd.read_csv(sales_file)
calendar_df = pd.read_csv(calendar_file)
prices_df = pd.read_csv(prices_file)

print(f'Sales data shape: {sales_df.shape}')
print(f'Calendar data shape: {calendar_df.shape}')
print(f'Prices data shape: {prices_df.shape}')

## Part 2: Exploratory Data Analysis (EDA)

In [ ]:
# Display basic info
print('=' * 60)
print('SALES DATA - First few rows')
print('=' * 60)
print(sales_df.head())
print(f'\nData shape: {sales_df.shape}')
print(f'Columns: product_id, store_id, category, dept, state, + 1913 day columns (d_1 to d_1913)')

In [ ]:
print('\n' + '=' * 60)
print('CALENDAR DATA')
print('=' * 60)
print(calendar_df.head(20))
print(f'\nCalendar shape: {calendar_df.shape}')
print(f'Date range: {calendar_df["date"].min()} to {calendar_df["date"].max()}')
print(f'\nEvents in calendar:')
print(calendar_df['event_name_1'].value_counts().head(10))

In [ ]:
# Unique stores and products
print('\n' + '=' * 60)
print('DATA STRUCTURE ANALYSIS')
print('=' * 60)

print(f'\nUnique stores: {sales_df["store_id"].nunique()}')
print(f'Stores: {sorted(sales_df["store_id"].unique())}')
print(f'\nUnique items: {sales_df["item_id"].nunique()}')
print(f'\nCategories: {sales_df["cat_id"].nunique()}')
print(f'Departments: {sales_df["dept_id"].nunique()}')
print(f'States: {sales_df["state_id"].nunique()}')

In [ ]:
# Products per store
products_per_store = sales_df.groupby('store_id')['item_id'].nunique()
print('\n' + '=' * 60)
print('PRODUCTS PER STORE')
print('=' * 60)
print(products_per_store)
print(f'\nTotal unique products: {sales_df["item_id"].nunique()}')

## Part 3: Focus on Store 1 (Single Store Analysis)

In [ ]:
# Filter for Store 1 only
STORE_ID = 1
store_1_sales = sales_df[sales_df['store_id'] == STORE_ID].copy()

print(f'\nStore {STORE_ID} Analysis:')
print('=' * 60)
print(f'Shape: {store_1_sales.shape}')
print(f'Products in store: {store_1_sales["item_id"].nunique()}')
print(f'Categories: {store_1_sales["cat_id"].nunique()}')
print(f'Departments: {store_1_sales["dept_id"].nunique()}')

In [ ]:
# Convert wide format (d_1, d_2, ..., d_1913) to long format
def pivot_sales_to_timeseries(sales_data, calendar_data):
    """
    Convert wide format sales data to long format time series.
    Input: columns [store_id, item_id, d_1, d_2, ..., d_1913]
    Output: DataFrame with [date, item_id, sales]
    """
    day_cols = [col for col in sales_data.columns if col.startswith('d_')]
    timeseries_data = []
    
    for idx, row in sales_data.iterrows():
        item_id = row['item_id']
        
        for day_col in day_cols:
            day_num = int(day_col.split('_')[1])
            sales_qty = row[day_col]
            
            timeseries_data.append({
                'item_id': item_id,
                'day_num': day_num,
                'sales': sales_qty
            })
    
    ts_df = pd.DataFrame(timeseries_data)
    
    # Merge with calendar to get dates
    calendar_sorted = calendar_data[['d', 'date']].drop_duplicates()
    calendar_sorted.columns = ['day_num', 'date']
    
    ts_df = ts_df.merge(calendar_sorted, on='day_num', how='left')
    ts_df['date'] = pd.to_datetime(ts_df['date'])
    
    return ts_df[['date', 'item_id', 'sales']].sort_values('date')

print('Converting to time series format...')
ts_df = pivot_sales_to_timeseries(store_1_sales, calendar_df)
print(f'Time series shape: {ts_df.shape}')
print(f'Date range: {ts_df["date"].min()} to {ts_df["date"].max()}')
print(f'Unique items: {ts_df["item_id"].nunique()}')

In [ ]:
# Sales statistics
print('\n' + '=' * 60)
print('SALES STATISTICS')
print('=' * 60)
print('\nOverall sales statistics:')
print(ts_df['sales'].describe())

zero_sales = (ts_df['sales'] == 0).sum()
total_records = len(ts_df)
zero_pct = (zero_sales / total_records) * 100
print(f'\nZero sales records: {zero_sales} ({zero_pct:.2f}%)')

print('\nSales distribution by item:')
print(ts_df.groupby('item_id')['sales'].agg(['count', 'sum', 'mean', 'max']).head(10))

In [ ]:
# Top products visualization
top_items = ts_df.groupby('item_id')['sales'].sum().nlargest(10)

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

top_items.plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Top 10 Products by Total Sales (Store 1)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Item ID')
axes[0].set_ylabel('Total Sales')
axes[0].grid(axis='y', alpha=0.3)

ts_df['sales'].hist(bins=50, ax=axes[1], color='steelblue', edgecolor='black')
axes[1].set_title('Distribution of Daily Sales (Store 1)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Daily Sales Quantity')
axes[1].set_ylabel('Frequency')
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

In [ ]:
# Time series trends for top 5 products
top_5_items = ts_df.groupby('item_id')['sales'].sum().nlargest(5).index

fig, axes = plt.subplots(5, 1, figsize=(14, 12))

for idx, item_id in enumerate(top_5_items):
    item_data = ts_df[ts_df['item_id'] == item_id]
    axes[idx].plot(item_data['date'], item_data['sales'], linewidth=1.5, color='steelblue')
    axes[idx].fill_between(item_data['date'], item_data['sales'], alpha=0.3, color='steelblue')
    axes[idx].set_title(f'Item {item_id} - Daily Sales Over Time', fontweight='bold')
    axes[idx].set_ylabel('Sales')
    axes[idx].grid(alpha=0.3)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# Seasonality analysis
ts_df['day_of_week'] = ts_df['date'].dt.day_name()
ts_df['month'] = ts_df['date'].dt.month

dow_sales = ts_df.groupby('day_of_week')['sales'].mean().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

dow_sales.plot(kind='bar', ax=axes[0], color='coral')
axes[0].set_title('Average Sales by Day of Week', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Day of Week')
axes[0].set_ylabel('Average Daily Sales')
axes[0].grid(axis='y', alpha=0.3)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45)

month_sales = ts_df.groupby('month')['sales'].mean()
month_sales.plot(kind='bar', ax=axes[1], color='lightgreen')
axes[1].set_title('Average Sales by Month', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Average Daily Sales')
axes[1].grid(axis='y', alpha=0.3)
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()

## Part 4: Data Preprocessing for Prophet

In [ ]:
# Select top product for initial training
TOP_ITEM_ID = top_5_items[0]

product_data = ts_df[ts_df['item_id'] == TOP_ITEM_ID].copy()
product_data = product_data.sort_values('date')

# Prepare for Prophet
prophet_df = product_data[['date', 'sales']].copy()
prophet_df.columns = ['ds', 'y']
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

print(f'Selected Product for Training: Item {TOP_ITEM_ID}')
print('=' * 60)
print(f'Dataset shape: {prophet_df.shape}')
print(f'Date range: {prophet_df["ds"].min()} to {prophet_df["ds"].max()}')
print(f'\nFirst 10 rows:')
print(prophet_df.head(10))
print(f'\nSales statistics:')
print(prophet_df['y'].describe())

In [ ]:
# Handle missing values
prophet_df['y'] = prophet_df['y'].fillna(method='ffill').fillna(0)

# Add holidays from calendar
holidays_df = calendar_df[calendar_df['event_name_1'].notna()].copy()
holidays_df = holidays_df[['date', 'event_name_1']].rename(
    columns={'date': 'ds', 'event_name_1': 'holiday'}
)
holidays_df['ds'] = pd.to_datetime(holidays_df['ds'])

print(f'Holidays to include: {len(holidays_df)}')
print(holidays_df.head(10))

## Part 5: Prophet Model Training

In [ ]:
print('\n' + '=' * 60)
print('PROPHET MODEL TRAINING')
print('=' * 60)

model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode='additive',
    interval_width=0.95,
    holidays=holidays_df
)

print(f'\nModel configuration:')
print(f'  Yearly seasonality: True')
print(f'  Weekly seasonality: True')
print(f'  Seasonality mode: Additive')
print(f'  Interval width: 95%')

print(f'\nTraining on {len(prophet_df)} observations...')
model.fit(prophet_df)
print('Model trained successfully!')

In [ ]:
# Generate forecast for 30 days
FORECAST_DAYS = 30
future = model.make_future_dataframe(periods=FORECAST_DAYS)

print(f'\nForecast period: {FORECAST_DAYS} days')
print(f'Future dataframe shape: {future.shape}')
print(f'Last training date: {prophet_df["ds"].max()}')
print(f'Last forecast date: {future["ds"].max()}')

In [ ]:
# Predict
print('\nGenerating forecast...')
forecast = model.predict(future)
print('Forecast generated successfully!')

print(f'\nForecast (last 35 rows - includes training + future):')
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(35))

In [ ]:
# Visualization
fig = model.plot(forecast, figsize=(14, 8))
plt.title(f'Prophet Forecast - Item {TOP_ITEM_ID} (30-Day Forecast with 95% CI)', 
          fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.tight_layout()
plt.show()

In [ ]:
# Components plot
fig = model.plot_components(forecast, figsize=(14, 10))
plt.tight_layout()
plt.show()

## Part 6: Model Evaluation

In [ ]:
# Calculate metrics on historical data
forecast_hist = forecast[forecast['ds'] <= prophet_df['ds'].max()].copy()
actual_hist = prophet_df.copy()

eval_df = actual_hist.merge(forecast_hist[['ds', 'yhat']], on='ds', how='left')

mape = mean_absolute_percentage_error(eval_df['y'], eval_df['yhat'])
mae = np.mean(np.abs(eval_df['y'] - eval_df['yhat']))
rmse = np.sqrt(np.mean((eval_df['y'] - eval_df['yhat'])**2))

print('\n' + '=' * 60)
print('MODEL EVALUATION METRICS')
print('=' * 60)
print(f'\nMAPE (Mean Absolute Percentage Error): {mape:.2f}%')
print(f'MAE (Mean Absolute Error): {mae:.2f} units')
print(f'RMSE (Root Mean Squared Error): {rmse:.2f} units')
print(f'Average actual sales: {eval_df["y"].mean():.2f} units')
print(f'Average forecast: {eval_df["yhat"].mean():.2f} units')

In [ ]:
# Actual vs Predicted (last 90 days)
last_90_days = eval_df.tail(90)

fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(last_90_days['ds'], last_90_days['y'], label='Actual', linewidth=2, color='blue')
ax.plot(last_90_days['ds'], last_90_days['yhat'], label='Forecast', linewidth=2, color='red', linestyle='--')
ax.fill_between(last_90_days['ds'], last_90_days['y'], last_90_days['yhat'], 
                 alpha=0.2, color='gray')
ax.set_title(f'Actual vs Predicted Sales - Last 90 Days (Item {TOP_ITEM_ID})', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Sales')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Residuals analysis
residuals = eval_df['y'] - eval_df['yhat']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(eval_df['ds'], residuals, linewidth=1, alpha=0.7, color='darkblue')
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0].fill_between(eval_df['ds'], residuals, 0, alpha=0.2, color='darkblue')
axes[0].set_title('Residuals Over Time', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Residual')
axes[0].grid(alpha=0.3)

axes[1].hist(residuals, bins=50, edgecolor='black', color='steelblue')
axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1].set_title('Distribution of Residuals', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Residual')
axes[1].set_ylabel('Frequency')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Part 7: Future Forecast Summary

In [ ]:
# Future forecast (next 30 days)
future_forecast = forecast[forecast['ds'] > prophet_df['ds'].max()][['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
future_forecast['yhat'] = future_forecast['yhat'].clip(lower=0)
future_forecast.columns = ['Date', 'Predicted_Sales', 'Lower_Bound', 'Upper_Bound']

print('\n' + '=' * 60)
print(f'FORECAST FOR NEXT {FORECAST_DAYS} DAYS - Item {TOP_ITEM_ID}')
print('=' * 60)
print(future_forecast.to_string(index=False))

print(f'\nForecast Summary:')
print(f'  Total predicted sales (30 days): {future_forecast["Predicted_Sales"].sum():.0f} units')
print(f'  Average daily sales: {future_forecast["Predicted_Sales"].mean():.2f} units')
print(f'  Max predicted sales: {future_forecast["Predicted_Sales"].max():.2f} units')
print(f'  Min predicted sales: {future_forecast["Predicted_Sales"].min():.2f} units')

## Part 8: Training Multiple Products

In [ ]:
def train_prophet_for_product(item_id, ts_data, holidays_data, forecast_days=30):
    """
    Train Prophet for a single product.
    Returns dict with model, forecast, and MAPE.
    """
    item_data = ts_data[ts_data['item_id'] == item_id].copy()
    item_data = item_data.sort_values('date')
    
    prophet_input = item_data[['date', 'sales']].copy()
    prophet_input.columns = ['ds', 'y']
    prophet_input['ds'] = pd.to_datetime(prophet_input['ds'])
    prophet_input['y'] = prophet_input['y'].fillna(method='ffill').fillna(0)
    
    if len(prophet_input) < 30:
        return None
    
    try:
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            seasonality_mode='additive',
            interval_width=0.95,
            holidays=holidays_data
        )
        model.fit(prophet_input)
        
        future = model.make_future_dataframe(periods=forecast_days)
        forecast = model.predict(future)
        
        forecast_hist = forecast[forecast['ds'] <= prophet_input['ds'].max()].copy()
        mape = mean_absolute_percentage_error(prophet_input['y'], forecast_hist['yhat'])
        
        return {
            'item_id': item_id,
            'model': model,
            'forecast': forecast,
            'mape': mape,
            'n_observations': len(prophet_input)
        }
    except Exception as e:
        print(f'Error for item {item_id}: {e}')
        return None

print('Function defined: train_prophet_for_product()')

In [ ]:
# Train for top 5 products
print('\n' + '=' * 60)
print('TRAINING MODELS FOR TOP 5 PRODUCTS')
print('=' * 60)

models_dict = {}

for item_id in top_5_items:
    print(f'\nTraining for item {item_id}...')
    result = train_prophet_for_product(item_id, ts_df, holidays_df, forecast_days=30)
    
    if result:
        models_dict[item_id] = result
        print(f'  Success - MAPE: {result["mape"]:.2f}%, Observations: {result["n_observations"]}')

print(f'\nTrained {len(models_dict)} models successfully')

In [ ]:
# Summary of trained models
print('\n' + '=' * 60)
print('TRAINED MODELS SUMMARY')
print('=' * 60)

summary_data = []
for item_id, result in models_dict.items():
    summary_data.append({
        'Item ID': item_id,
        'MAPE (%)': f"{result['mape']:.2f}",
        'Observations': result['n_observations'],
        'Status': 'Ready'
    })

summary_df = pd.DataFrame(summary_data)
print('\n' + summary_df.to_string(index=False))

avg_mape = np.mean([r['mape'] for r in models_dict.values()])
print(f'\nAverage MAPE: {avg_mape:.2f}%')

## Part 9: Save Models

In [ ]:
import pickle
import json
from pathlib import Path

MODELS_DIR = Path('../models')
MODELS_DIR.mkdir(exist_ok=True)

print('\n' + '=' * 60)
print('SAVING TRAINED MODELS')
print('=' * 60)

for item_id, result in models_dict.items():
    model_path = MODELS_DIR / f'prophet_item_{item_id}.pkl'
    with open(model_path, 'wb') as f:
        pickle.dump(result['model'], f)
    print(f'Saved: {model_path.name}')

# Save metadata
metadata = {
    'store_id': STORE_ID,
    'trained_items': list(models_dict.keys()),
    'model_count': len(models_dict),
    'training_date': datetime.now().isoformat(),
    'dataset_info': {
        'total_observations': len(ts_df),
        'total_items': ts_df['item_id'].nunique()
    }
}

metadata_path = MODELS_DIR / f'metadata_store_{STORE_ID}.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f'Saved: {metadata_path.name}')
print('\nAll models saved successfully!')

In [ ]:
# Verify by loading a saved model
print('\nVerifying saved models...')
test_item_id = list(models_dict.keys())[0]
test_model_path = MODELS_DIR / f'prophet_item_{test_item_id}.pkl'

with open(test_model_path, 'rb') as f:
    loaded_model = pickle.load(f)

print(f'Successfully loaded model for item {test_item_id}')
print(f'Model type: {type(loaded_model)}')
print('All checks passed!')

## Summary

Completed:
1. EDA on M5 Walmart dataset (10 stores, 30K+ products, 1913 days)
2. Filtered to Store 1 (3K products for focused analysis)
3. Converted wide format to time series
4. Trained Prophet model on top product with 30-day forecast
5. Evaluated with MAPE, MAE, RMSE metrics
6. Scaled to train 5 products
7. Saved all models for production use

Next: Integrate into Django for API endpoints